In [23]:
from cosmikyu import gan, config, model
from cosmikyu import nn as cnn
import numpy as np
import os
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import torch
import mlflow
import torchsummary

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from torch import nn
nn.parallel.data_parallel??

In [33]:
shape = (4,128,128)
unet_gen = model.UNET_Generator(
    shape,
    nconv_layer=4,
    nconv_fc=64,
    ngpu=1,
    kernal_size=5,
    stride=2,
    padding=2,
    output_padding=1,
    normalize=True,
    activation=None,
).to(device="cpu")

unet_disc = model.UNET_Discriminator(
    shape=shape,
    nconv_layer=4,
    nconv_fc=64,
    ngpu=1,
    kernal_size=5,
    stride=2,
    padding=2,
    normalize=True,
).to(device="cpu")

print("====dc_wgp======")
torchsummary.summary(unet_disc, shape, device="cpu")
torchsummary.summary(unet_gen, shape, device="cpu")
#print(type(unet_disc.model))
#torchsummary.summary(DCGAN_WGP.generator, (latent_dim,))

====dc_wgp======
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           6,464
         LeakyReLU-2           [-1, 64, 64, 64]               0
            Conv2d-3          [-1, 128, 32, 32]         204,928
       BatchNorm2d-4          [-1, 128, 32, 32]             256
         LeakyReLU-5          [-1, 128, 32, 32]               0
            Conv2d-6          [-1, 256, 16, 16]         819,456
       BatchNorm2d-7          [-1, 256, 16, 16]             512
         LeakyReLU-8          [-1, 256, 16, 16]               0
            Conv2d-9            [-1, 512, 8, 8]       3,277,312
      BatchNorm2d-10            [-1, 512, 8, 8]           1,024
        LeakyReLU-11            [-1, 512, 8, 8]               0
           Conv2d-12            [-1, 512, 4, 4]       6,554,112
      BatchNorm2d-13            [-1, 512, 4, 4]           1,024
        LeakyReLU-14  

In [45]:
torchsummary.summary??

In [ ]:
unet_disc = model.UNET_Discriminator

In [29]:
STanh = cnn.ScaledTanh(15., 2./15.)
latent_dim = 256
shape = (5,128,128)

DCGAN_WGP = gan.DCGAN_WGP("sehgal_dcganwgp", shape, latent_dim, cuda=False, nconv_fcgen=64,
                              nconv_fcdis=64, ngpu=4, nconv_layer_gen=5, nconv_layer_disc=5, kernal_size=5, stride=2,
                                                        padding=2, output_padding=1, gen_act=STanh)

print("====cosmogan_wgp======")
torchsummary.summary(DCGAN_WGP.discriminator, shape)
torchsummary.summary(DCGAN_WGP.generator, (latent_dim,))
del DCGAN_WGP

====cosmogan_wgp======
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           8,064
         LeakyReLU-2           [-1, 64, 64, 64]               0
            Conv2d-3          [-1, 128, 32, 32]         204,928
         LeakyReLU-4          [-1, 128, 32, 32]               0
            Conv2d-5          [-1, 256, 16, 16]         819,456
         LeakyReLU-6          [-1, 256, 16, 16]               0
            Conv2d-7            [-1, 512, 8, 8]       3,277,312
         LeakyReLU-8            [-1, 512, 8, 8]               0
            Conv2d-9           [-1, 1024, 4, 4]      13,108,224
        LeakyReLU-10           [-1, 1024, 4, 4]               0
          Reshape-11                [-1, 16384]               0
           Linear-12                    [-1, 1]          16,385
Total params: 17,434,369
Trainable params: 17,434,369
Non-trainable params: 0
--